In [1]:
import locallib.eyeutil as eyeutil
import locallib.crypto as crypto


In [10]:
# Data manually acquired
alphabetRaw = {
  "A": ( "203", "111" ),
  "D": ( "002", "111" ),
  "E": ( "000", "111" ),
  "G": ( "022", "430" ),
  "H": ( "234", "233" ),
  "I": ( "000", "000" ),
  "J": ( "203", "403" ),
  "K": ( "031", "030" ),
  "L": ( "203", "403" ),
  "M": ( "111", "000" ),
  "N": ( "000", "340" ),
  "O": ( "302", "444" ),
  "P": ( "444", "324" ),
  "R": ( "232", "111" ),
  "S": ( "323", "111" ),
  "T": ( "022", "300" ),
  "U": ( "210", "032" ),
  "V": ( "000", "420" ),
  "Y": ( "000", "222" )
}

def get_grid(rot, ind, v):
  # Standard trigram, 1 in far right
  if not rot:
    if v == '0': return str(ind * 3 + 2)
    elif v == '1': return str(ind * 3 + 3)
    elif v == '2': return str(ind * 3 + 3)
    elif v == '3': return str(ind * 3 + 1)
    elif v == '4': return str(ind * 3 + 1)
  
  # perpendicular trigram, 1 in far back
  else:
    if v == '0': return str(1 * 3 + (ind + 1))
    elif v == '1': return str(0 * 3 + (ind + 1))
    elif v == '2': return str(0 * 3 + (ind + 1))
    elif v == '3': return str(2 * 3 + (ind + 1))
    elif v == '4': return str(2 * 3 + (ind + 1))


def line_set_value(g0, g1):
  # Use grid points as keys
  p0 = g0 if g0 < g1 else g1
  p1 = g1 if g0 < g1 else g0
  return str(p0) + ',' + str(p1)


# Get list of all spanning liness
longLines = { }
for i in range(3):
  longLines[line_set_value( 1 + i,      7 + i     )] = [ line_set_value( 1 + i,      4 + i ),      line_set_value( 4 + i,      7 + i ) ]
  longLines[line_set_value( 1 + i * 3,  3 + i * 3 )] = [ line_set_value( 1 + i * 3,  2 + i * 3 ),  line_set_value( 2 + i * 3,  3 + i * 3 ) ]
longLines[line_set_value( 1, 9 )] = [ line_set_value( 1, 5 ),   line_set_value( 5, 9 ) ]
longLines[line_set_value( 3, 7 )] = [ line_set_value( 3, 5 ),   line_set_value( 5, 7 ) ]


def remove_full_lines(lines):
  # Remove double length lines
  newLines = set()
  for line in lines:
    if line in longLines:
      newLines.add(longLines[line][0])
      newLines.add(longLines[line][1])
    else:
      newLines.add(line)
  return newLines


def get_lines(bt):
  # Calculate which points are hidden
  h = [ [], [] ]
  for i in range(2):
    for o in range(3):
      h[i].append(bt[i][o] == '1' or bt[i][o] == '4')

  # Calculate grid points of both eyes
  g = [ [], [] ]
  for i in range(2):
    for o in range(3):
      g[i].append(get_grid(i == 1, o, bt[i][o]))

  # Create a set of all lines
  lines = set()
  for i in range(2):
    if not h[i][0] and not h[i][1]: lines.add(line_set_value(g[i][0], g[i][1]))
    if not h[i][1] and not h[i][2]: lines.add(line_set_value(g[i][1], g[i][2]))
    if not h[i][0] and h[i][1] and not h[i][2]: lines.add(line_set_value(g[i][0], g[i][2]))
  
  # Remove full lines and return
  return remove_full_lines(lines)


In [12]:
# Loop over all bitrigrams
allBitrigramLines = { }
allLinesBitrigrams = { }
for i in range(15625):

  # Only allow valid trigrams
  based = eyeutil.conv_tri_to_triflat_open(i, 5).zfill(6)
  tri1 = based[0:3]
  tri2 = based[3:6]
  if not eyeutil.check_tri_valid(tri1) or not eyeutil.check_tri_valid(tri2): continue

  # Get lines for each bitrigram
  bt = [ tri1, tri2 ]
  lines = get_lines(bt)
  allBitrigramLines[based] = lines

  # Get bitrigram for each lines
  key = frozenset(lines)
  if key not in allLinesBitrigrams:
    allLinesBitrigrams[key] = [ ]
  allLinesBitrigrams[key].append(based)

# Get all bitrigrams that are equivalent
alphabetAll = { }
for letter in alphabetRaw:
  lines = get_lines(alphabetRaw[letter])
  key = frozenset(lines)
  bitrigrams = allLinesBitrigrams[key]
  alphabetAll[letter] = bitrigrams



In [13]:
# Show all alphabet sets
for bt in allBitrigramLines:
  print("\nBi-Trigram: " + bt)
  print(allBitrigramLines[bt])



Bi-Trigram: 000000
{'2,5', '5,8', '4,5', '5,6'}

Bi-Trigram: 000001
{'2,5', '5,8', '4,5'}

Bi-Trigram: 000002
{'2,5', '5,8', '3,5', '4,5'}

Bi-Trigram: 000003
{'2,5', '5,8', '5,9', '4,5'}

Bi-Trigram: 000004
{'2,5', '5,8', '4,5'}

Bi-Trigram: 000010
{'2,5', '5,8', '5,6', '4,5'}

Bi-Trigram: 000011
{'2,5', '5,8'}

Bi-Trigram: 000012
{'2,5', '5,8', '3,4'}

Bi-Trigram: 000013
{'2,5', '5,8', '4,9'}

Bi-Trigram: 000014
{'2,5', '5,8'}

Bi-Trigram: 000020
{'2,5', '5,8', '2,6', '2,4'}

Bi-Trigram: 000021
{'2,5', '5,8', '2,4'}

Bi-Trigram: 000022
{'2,5', '5,8', '2,4', '2,3'}

Bi-Trigram: 000023
{'2,5', '5,8', '2,9', '2,4'}

Bi-Trigram: 000024
{'2,5', '5,8', '2,4'}

Bi-Trigram: 000030
{'2,5', '5,8', '4,8', '6,8'}

Bi-Trigram: 000031
{'2,5', '5,8', '4,8'}

Bi-Trigram: 000032
{'2,5', '5,8', '3,8', '4,8'}

Bi-Trigram: 000033
{'2,5', '5,8', '8,9', '4,8'}

Bi-Trigram: 000034
{'2,5', '5,8', '4,8'}

Bi-Trigram: 000040
{'2,5', '5,8', '5,6', '4,5'}

Bi-Trigram: 000041
{'2,5', '5,8'}

Bi-Trigram: 000042


In [14]:
# Show all alphabet sets
for letter in alphabetAll:
  print("\nLetter " + letter + ":")
  print(alphabetAll[letter])


Letter A:
['011302', '014302', '041302', '044302', '101302', '103102', '103302', '104302', '110302', '111302', '112302', '113302', '114302', '121302', '124302', '131302', '134302', '140302', '141302', '142302', '143302', '144302', '201301', '201302', '201304', '203011', '203014', '203041', '203044', '203101', '203102', '203104', '203110', '203111', '203112', '203113', '203114', '203121', '203124', '203131', '203134', '203140', '203141', '203142', '203143', '203144', '203211', '203214', '203241', '203244', '203301', '203302', '203304', '203311', '204301', '204302', '204304', '211302', '213011', '213014', '213041', '213044', '213101', '213102', '213104', '213110', '213111', '213112', '213113', '213114', '213121', '213124', '213131', '213134', '213140', '213141', '213142', '213143', '213144', '213211', '213214', '213241', '213244', '213301', '213302', '213304', '213311', '214302', '241302', '243011', '243014', '243041', '243044', '243101', '243102', '243104', '243110', '243111', '243112'

In [15]:
# Helper function to match a bitrigram to a letter
def get_letter(bt):
  for letter in alphabetAll:
    if bt in alphabetAll[letter]:
      return letter
  return None

# Calculate pecentage
def check_letters(paired):
  total = sum([ len(msg) for msg in paired ])
  count = 0
  for msg in paired:
    for bt in msg:
      letter = get_letter(bt)
      if letter is not None:
        count += 1
  return count, total, 100 * count / total


In [17]:
# Pair up each message odd / even
paired = []
for msg in eyeutil.msgs_eye_triflat:
  pairedMsg = []
  for i in range(0, len(msg), 2):
    tri1 = msg[i]
    if i >= len(msg) - 1: tri2 = '111'
    else: tri2 = msg[i + 1]
    pairedMsg.append(tri1 + tri2)
  paired.append(pairedMsg)

# Pair up east / west
# paired = []
# for i in range(4):
#   pairedMsg = []
#   msg1 = eyeutil.msgs_eye_triflat[i * 2]
#   msg2 = eyeutil.msgs_eye_triflat[i * 2 + 1]
#   for o in range(min(len(msg1), len(msg2))):
#     pairedMsg.append(msg1[o] + msg2[o])
#   paired.append(pairedMsg)

# Output paired info
for i in range(len(paired)):
  analysis = check_letters([ paired[i] ])
  print(i, analysis)
print("All", check_letters(paired))

0 (4, 50, 8.0)
1 (5, 52, 9.615384615384615)
2 (3, 59, 5.084745762711864)
3 (3, 51, 5.882352941176471)
4 (10, 69, 14.492753623188406)
5 (6, 62, 9.67741935483871)
6 (6, 60, 10.0)
7 (10, 60, 16.666666666666668)
8 (6, 57, 10.526315789473685)
All (53, 520, 10.192307692307692)
